# Introduction to using ParticleSpy

This is an example notebook for performing particle analysis using ParticleSpy.

This notebook takes a HAADF image of nanoparticles, allows the user to determine the best segmentation method and then performs analysis, producing a plot of particle areas.

Below, we load an image using Hyperspy.

In [ ]:
import hyperspy.api as hs
import ParticleSpy.api as ps

filename = "data/SiO2 HAADF Image.hspy"
haadf = hs.load(filename)

%matplotlib inline
haadf.plot()

Next, we can examine the image and determine the optimum segmentation procedure using the segmentation GUI. Experiment with changing the threshold algorithm and the application of watershedding to obtain the optimum labels. Once happy, click "Get Params" to print the current parameters.

In [ ]:
ps.

Next, we use the parameters saved when last pressing Update in the GUI to construct a parameter dictionary and use this to perform the segmentation.

In [ ]:
params = 
params.
params.
particles = 

Let's interrogate the particle data a little bit. From an initial image we get a number of particle properties calculated automatically.

In [ ]:
print()
print()

It is then possible to plot a histogram of particle area.

In [ ]:
particles.

It is also possible to plot a scatter plot of two properties. In this instance, plotting 'intensity' vs area.

In [ ]:
particles.

Properties are one of the key parts of ParticleSpy. Not only can you plot them, it's possible to cluster different particles based on their properties. For example, clustering of the SiO2 particles based on both area and 'intensity':

In [ ]:
particles_clustered = particles.
ps.

## ParticleSpy with EDS data

If you have EDS data collected simultaneously with an image, it is possible to extract maps, spectra and the composition from each segmented particle. Firstly, load the EDS spectrum image.

In [ ]:
eds_filename = "data/SiO2 EDS Spectrum Image.hspy"
eds = hs.load(eds_filename)

Next, put the HAADF image and EDS spectrum image together in a list to make one acquisition.

In [ ]:
ac = [haadf,eds]

At this point, we need to set the parameters for the EDS analysis. This can be done by taking our previously used parameters object adding parameters with generate_eds().

In [ ]:
params.

Now, we need to run ParticleAnalysis again in order to get the accompanying EDS data.

In [ ]:
particles2 = ps.ParticleAnalysis(ac,params)

The extracted maps of each element can be accessed from the particle list from maps['element'].

In [ ]:
particles2.list[0].maps['Si'].as_signal2D([0,1]).plot()

The composition of each particle can be accessed from the particle list using list[particle].composition.

In [ ]:
particles2.list[0].composition

We can plot a radial profile of the particle intensity from both the image and our elemental maps.

In [ ]:
rp = ps.

In [ ]:
rp2 = ps.

## ParticleSpy with EELS data

ParticleSpy does not automatically process EELS data other than to store a sum spectrum for each particle. This gives us an opportunity to explore adding properties to particles though, woohoo! Let's start by loading EELS data for new particles.

In [ ]:
ac2 = [hs.load(r'C:\Users\qzo13262\Desktop\Hyperspy Workshop\data/ADF Image.dm4'),hs.load(r'C:\Users\qzo13262\Desktop\Hyperspy Workshop\data/EELS Spectrum Image (high-loss).dm4'),hs.load(r'C:\Users\qzo13262\Desktop\Hyperspy Workshop\data/EELS Spectrum Image (low-loss).dm4')]

For this dataset it is difficult to use automated segmentation routines to segment the particles. Therefore, we're going to load a manually created mask.

In [ ]:
mask = hs.load(r'C:\Users\qzo13262\Desktop\Hyperspy Workshop\data/EELS mask.tif')

In [ ]:
mask.plot()

Now, the loaded binary mask can be given as the mask argument in ParticleAnalysis.

In [ ]:
particles3 = ps.ParticleAnalysis(ac2,params,mask=)

In [ ]:
particles3.list[0].spectrum['EELS-HL'].plot()

We can now write a function to analyse the EELS spectrum of each particle and add certain properties from this spectrum to each particle. I have written a function that fits Gaussians to the white lines in the Co L edge, in a similar manner to the end of the EELS tutorial. At the end of the function, we'll add some of the results of the fit as particle properties.

In [ ]:
def co_eels_analysis(p):
    p.spectrum['EELS-LL'].align_zero_loss_peak(also_align=[p.spectrum['EELS-HL']])
    p.spectrum['EELS-HL'].crop_signal1D(750.0,850.0)
    spec = p.spectrum['EELS-HL'].remove_background(signal_range=(750.0,780.0),fast=False)
    m = spec.create_model(ll=p.spectrum['EELS-LL'], auto_background=False)
    m.set_signal_range((775.0,810.0))
    g1 = hs.model.components1D.GaussianHF()
    m.append(g1)
    m.fit_component(g1,(780.0,790.0))
    g2 = hs.model.components1D.GaussianHF()
    m.append(g2)
    m.fit_component(g2,(792.0,802.0))
    m.plot(plot_components=True)
    g3 = hs.model.components1D.HeavisideStep()
    m.append(g3)
    g3.n.value = g1.centre.value
    g3.n.bmin = g1.centre.value - 0.5
    g3.n.bmax = g1.centre.value + 0.5
    g3.n.bounded = True
    g4 = hs.model.components1D.HeavisideStep()
    m.append(g4)
    g4.n.value = g2.centre.value
    g4.n.bmin = g2.centre.value - 0.5
    g4.n.bmax = g2.centre.value + 0.5
    g4.n.bounded = True
    m.multifit()
    
    g1_g2_ratio = (g1.height.value*abs(g1.fwhm.value))/(g2.height.value*abs(g2.fwhm.value))
    p.
    g1_centre = g1.centre.value
    p.
    g1_g2_distance = g2.centre.value-g1.centre.value
    p.

The next step is to iteratively use this function on each particle.

In [ ]:
for p in particles3.list:
    co_eels_analysis(p)

It's now possible to use the plotting functions to plot the new properties from the EELS data.

In [ ]:
particles3.plot()